<center>

<h1>Building Chatbot for Interactive and Interactive Recipe recommendation</h1>
<h3>MSDS 560 - Natural Language PRocessing Project</h3>
<h3>Group A - Bradford Patton and Christin Oguno</h3>

</center>

<h2>Project Description</h2>

Specific Aims: 
    1. Matching the recipes based on user preferences. 

A large dataset of recipes is provided to build your NLP pipeline. 

Tasks:
1. Build a Chatbot that accepts user input in multiple iterations. 
2. Retrieve recipes based on user input and 
3. filter in the sub-sequent iteration.
Output.

In a iterative process, this chatbot will help a user gather recipes of their preferences. Final output is a list of recipes that satisfies the user criteria. 


YourRecipeBot.com is new start-up that looking to improve the user experience around recipe search. They have identified that typically users having difficult time find recipes right for their diet. 

This new company is building a new chatBot for interactive recipe search. The chatBot will interact with the user. It will take input about recipe preference and provide recommendation of recipes. User get the chance refine the search in subsequence interaction by putting new search criteria. And the chatBot again filter the recommendation. Over a brief interaction, the company hope that user will get what she is looking for. 

They need you help. 

They are using publicly available recipe databases for training and testing the model before moving to production level implementation. 

Relevant Article:

https://aclanthology.org/D19-1613/ 

## Load Dataset

In [ ]:
import pandas as pd
import numpy as np
recipe=pd.read_csv('/kaggle/input/recipe-dataset-over-2m/recipes_data.csv')

In [ ]:
recipe.head(2)

In [ ]:
recipe.ingredients[0]

Working with only a small version of the dataset

In [ ]:
recipe_small=recipe.head(500) 

In [ ]:
recipe_small.count()

In [ ]:
recipe_small.title.duplicated().tail(90)

In [ ]:
duplicated_titles = recipe_small[recipe_small.duplicated(subset='title', keep=False)]
print("Rows with Duplicated Titles:")
print(duplicated_titles)

In [ ]:
title_counts = recipe_small['title'].value_counts()
duplicated_title_counts = title_counts[title_counts > 1]
print("\nDuplicated Titles and Their Counts:")
print(duplicated_title_counts)

In [ ]:
recipe_small.NER[0]

In [ ]:
recipe_small.index

## *Using NER column because I want to match recipes based on key words which is that column

In [ ]:
indices = pd.Series(recipe_small.index, index = recipe_small['NER'])
indices

In [ ]:
recipe_small.shape

A recommendation system takes an input (words) or activities (online shopping searches) from you 

You want to get the RECIPES that contain these words

Based on similarity metrics- cosign similarity 


## Data Cleaning

In [ ]:
import nltk # natural language toolkit
nltk.download('all',  halt_on_error=False)
import spacy

from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup # for cleaning addidtional text from document 
import unicodedata

In [ ]:

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
nlp = spacy.load("en_core_web_sm") # Initialize Spacy for english language.
tokenizer = ToktokTokenizer() # Initialize the nltk tokenizer
stopword_list = nltk.corpus.stopwords.words('english') # populate the stop-word list from nltk.
stopword_list.remove('no')   # remove 'no' from the stop-word. Because no is important
stopword_list.remove('not')  # Similarly remove 'not' from stop-word. 

In [ ]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")  ## calling BeautifulSoup function to process text
    stripped_text = soup.get_text()  ## calling get_text to get the cleaned text
    return stripped_text   ## return the clean text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def remove_special_characters(text):
    text = re.sub('[^a-zA-Z0-9\s]', '', text)   ## Remove the characters other than a-z A-Z 0-9 and space. 
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) \
                                   if contraction_mapping.get(match) \
                                    else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

import re

def convert_func(matchobj):
    m =  matchobj.group(0)
    map = {'7': 'seven',
           '8': 'eight',
           '9': 'nine'}
    return map[m]
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")  ## calling BeautifulSoup function to process text
    stripped_text = soup.get_text()  ## calling get_text to get the cleaned text
    return stripped_text 

In [ ]:
def normalize_corpus(corpus, html_stripping= True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True):
    
    normalized_corpus = []
    
    for doc in corpus:
        
        if html_stripping:
            doc = strip_html_tags(doc)
        
        if accented_char_removal:
            doc = remove_accented_chars(doc)
            
        if contraction_expansion:
            doc = expand_contractions(doc)
            
        if text_lower_case:
            doc = doc.lower()
            
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        
        if text_lemmatization:
            doc = lemmatize_text(doc)
            
        if special_char_removal:
            doc = remove_special_characters(doc)  
            
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

#created the clean data // doing the work // but do we need more columns? 
This only cleaned NER column

In [ ]:
#type(recipes)

### Clean all necessary columns: NER, ingredients, directions, and title 

## Add all cleaned to columns to OG dataset

We need to take this cleaned columns and add them to our dataframe so we can use it for further processing steps

In [ ]:
recipe_small['cleaned_NER']= normalize_corpus(recipe_small['NER'])
recipe_small['cleaned_ingredients']= normalize_corpus(recipe_small['ingredients'])
recipe_small['cleaned_directions']= normalize_corpus(recipe_small['directions'])
recipe_small['cleaned_title']= normalize_corpus(recipe_small['title'])

In [ ]:
recipe_small.tail(2)

In [ ]:
recipe_small['new_title'] = (
    recipe_small['cleaned_title'] + recipe_small.groupby('cleaned_title')
    .cumcount()
    .astype(str)
    .replace('0', '') 
)

## Tfidf Vectorizing recommendation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(analyzer= 'word', ngram_range=(1,5), min_df=0.002, max_df=0.5, max_features=5000)

ngram range defines dictinoary
directions to ngram (1,5) and kept the max features the same 
next we will concatnate all the rows we need in our recommedation system

In [ ]:
recipe_small2= recipe_small['new_title']+""+recipe_small['cleaned_ingredients']+""+recipe_small['cleaned_directions']+""+recipe_small['cleaned_NER']


In [ ]:
tfidf_matrix = tfidf.fit_transform(recipe_small2)

In [ ]:
tfidf_matrix.shape

In [ ]:
#print(tfidf.vocabulary_)

## Step 3: Copmuting Pair-wise similarity of articles: once we get vectors compute the similarity between recipes - for EACH recipe we want to find the similarity score comparing with ALL the other recipes in the dataset = we will have the simlarity values of each recipe to EACH recipe 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_matrix=cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
similarity_matrix.shape

In [ ]:
#comparting the first row with the first row
similarity_matrix[0][0]

In [ ]:
print(recipe_small['cleaned_title'][0])
print(recipe_small['cleaned_NER'][1])
print(similarity_matrix[0][1])

## Accepting User Input:

## Recommending based on Title

In [ ]:
indices1 = pd.Series(recipe_small.index, index = recipe_small['new_title'])
indices1

The different values represent the different similarity values. We need to get the indexes for each of these so we can link it to the data to get actual recipe titles. 

In [ ]:
def recommendations(user_input, topk=10):
    idx=indices1[user_input]
    sim_values=similarity_matrix[idx]
    sim_values=list(enumerate(sim_values))
    sim_values=sorted(sim_values, key=lambda x:x[1], reverse=True)
    ## getting the titles by indices. 
    content_index=[i[0] for i in sim_values[1:topk+1]]
    recom = recipe_small[['new_title', 'ingredients', 'directions', 'link']].iloc[content_index, :]
    return recom

In [ ]:
#recommendations('log cabin toast')

In [ ]:
recommendations('bake bean')

In [ ]:
recommendations('skillet beef macaroni')

# creamy cheese crab dip

In [ ]:
recommendations('creamy cheese crab dip')

# The second monkey bread recipe

In [ ]:
recommendations('monkey bread1')

### HOW to Recommending by a random request/user_input not just using title as an index but the entire df: 
Since we are doing this based on index, we have to find the index of the user input 

## Word2vec reommendation system

In [ ]:
from gensim.models import Word2Vec 
from nltk.tokenize.toktok import ToktokTokenizer
import nltk # natural Langauge Toolkit - 
nltk.download('all',  halt_on_error=False)

In [ ]:
tokenizer = ToktokTokenizer()
tokenized_recipes=[]
for sentence in recipe_small2 :
    tokens=tokenizer.tokenize(sentence)
    tokenized_recipes.append(tokens)
model=Word2Vec(sentences=tokenized_recipes,vector_size=500,window=30, epochs=7)

In [ ]:
corpus_feature_w2v=[]

for recipe_tokens in tokenized_recipes:
    feature_vector=[model.wv[w] for w in recipe_tokens if w in model.wv]
    temp_features=sum(feature_vector)/len(recipe_tokens)
    corpus_feature_w2v.append(temp_features)
corpus_feature_w2v=np.array(corpus_feature_w2v)

corpus_feature_w2v.shape

In [ ]:
similarity_matrix= cosine_similarity(corpus_feature_w2v, corpus_feature_w2v)
similarity_matrix.shape

In [ ]:
def recommendations(user_input, topk=10):
    idx=indices1[user_input]
    sim_values=similarity_matrix[idx]
    sim_values=list(enumerate(sim_values))
    sim_values=sorted(sim_values, key=lambda x:x[1], reverse=True)
    content_index=[i[0] for i in sim_values[1:topk+1]]
    recom = recipe_small[['new_title', 'ingredients', 'directions', 'link']].iloc[content_index, :]
    return recom

#  old time bread pudding

In [ ]:
recommendations('old time bread pudding')

# baked beans

In [ ]:
#recommendations('bake bean')

# monkey bread

In [ ]:
recommendations('monkey bread')